# Chapter5: Streaming Deployment

>This notebook is based on the open-source project [wow-rag](https://github.com/datawhalechina/wow-rag) by Datawhale China.  
>I’ve adapted and annotated parts of it for personal learning and experimentation.

## 1. Introduction : What does "streaming deployment" mean?

Streaming deployment refers to  where responses from an LLM are generated and displayed incrementally (token by token or sentence by sentence) instead of waiting for the full output at once. 

This creates a more interactive user experience, especially for long or complex responses.

**Why do we need it?**

- Faster Feedback: Users see partial results immediately instead of waiting.

- Interactive Feel: It simulates a real-time conversation, making chatbots feel more responsive.

- Scalable UX: Especially useful in web interfaces where latency matters.

## 2. Preparation

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('API_KEY')

#base_url = "hhttps://api.openai.com/v1"  # We use openai's model here
chat_model = "gpt-4.1-nano-2025-04-14"   # We will be using cheaper model as im broke AF
emb_model = "text-embedding-3-small"

from openai import OpenAI
client = OpenAI(
    api_key = api_key,
    #base_url = base_url
)

from llama_index.llms.openai import OpenAI
llm = OpenAI(
    api_key = api_key,
    model = chat_model,
)

from llama_index.embeddings.openai import OpenAIEmbedding
embedding = OpenAIEmbedding(
    api_key = api_key,
    model = emb_model,
)
emb = embedding.get_text_embedding("Hellooo")



from llama_index.core import SimpleDirectoryReader,Document
documents = SimpleDirectoryReader(input_files=['./docs/example.txt']).load_data()


from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents,embed_model=embedding)

## 3.  Streaming Responses with `response_gen`


We're using `query_engine = index.as_query_engine(streaming=True)` to simulate a **real-time generation experience** — ideal for chatbot-like interfaces or interactive writing assistants.

###  Alternative Methods:
- **No Streaming**: Use `streaming=False` and wait for the entire result, then print.
- **FastAPI Backend**: Use a backend server to stream via `StreamingResponse` (e.g., for web apps).
- **LangChain**: An alternative to LlamaIndex, also supports streaming with various models.
- **Direct OpenAI API**: If using OpenAI's models, you can stream tokens via their `stream=True` parameter in `openai.ChatCompletion.create()`.

### 3.1 Construct query engine

In [3]:
query_engine = index.as_query_engine(
    streaming=True, 
    similarity_top_k=3,
    llm=llm)

### 3.2 Construct Stream reponse

In [ ]:

response_stream = query_engine.query("Please write a 100-word article on the employment prospects of AI majors") 
buffer = "" # Buffer for auto line wrap
for text in response_stream.response_gen: # Stream the output as it's generated
    buffer += text
    while "." in buffer:
        sentence, buffer = buffer.split(".", 1)
        print(sentence.strip() + ".\n")

The employment prospects for AI majors are promising, as the technology continues to expand across various industries.

AI specialists are in high demand for roles in healthcare, gaming, manufacturing, and content generation, where they develop and refine intelligent systems.

Responsible development and ethical considerations are increasingly prioritized, creating opportunities for experts in AI ethics and safety.

However, professionals must stay adaptable, as the field evolves rapidly with new applications and challenges.

Continuous learning and understanding of multimodal models, collaboration policies, and safety measures will be essential for AI majors seeking to thrive in this dynamic job market.



## 4. Streaming with `FastAPI`

### FastAPI: A Modern Web Framework for AI Backends


FastAPI is a high-performance web framework for building APIs with Python 3.7+, based on standard Python type hints.

It’s widely used to deploy machine learning models, including LLM-based services, by wrapping them in a RESTful API or WebSocket backend.

---

###  Why Use FastAPI?

Use FastAPI when you want to:

-  Build a full backend API for your LLM or vector search app  
-  Integrate with frontends or other services (e.g., JavaScript, React, mobile apps)  
-  Serve models or embeddings remotely, rather than keeping them embedded in notebooks  

---

###  When to Use FastAPI?

- Building a **production-grade** AI application  
- Integrating with a **frontend UI** (Streamlit, React, Vue, etc.)  
- Exposing your model to **external users or services**  
- **Scaling** to multiple users or concurrent requests  

---

###  Uvicorn :  ASGI web server for Python

**Uvicorn** is a lightning-fast **ASGI web server** for Python — it's what actually runs our **FastAPI** app when deployed.

###  Key Points:
- Uvicorn stands for **"Universal Interface for ASGI applications running on asyncio"**.
- It runs your FastAPI app by listening to HTTP requests and routing them to your Python code.
- Unlike older WSGI servers (like Flask uses), **ASGI** supports **asynchronous programming**, which is perfect for handling many concurrent requests — like a chatbot or LLM app.




### 4.1 Package Installation

In [11]:
# %pip install fastapi
# %pip install uvicorn

### 4.2 FastAPI Streaming Server Example 

This example shows how to set up a basic FastAPI server that streams responses from a language model  or query engine in real time using Server-Sent Events (SSE). It's useful for building chatbot backends or interactive web apps with live outputs.

The code includes:
- FastAPI app creation and CORS setup
- A background thread to run the server from a notebook
- A streaming endpoint (`/stream_chat`) to serve token-by-token responses

In [ ]:
# == Import required libraries ==

import uvicorn  # ASGI server used to run FastAPI apps
from fastapi import FastAPI  # Main web framework
from fastapi.middleware.cors import CORSMiddleware  # To allow cross-origin requests (e.g., frontend calls)
from fastapi.responses import StreamingResponse  # Allows returning streaming responses (like token-by-token LLM output)
import threading  # For running the server in a background thread (useful inside notebooks)

# == Initialize FastAPI app ==
app = FastAPI()

# == Allow all cross-origin requests (good for prototyping, but lock it down for production) ==
app.add_middleware(CORSMiddleware, allow_origins=["*"])

# == Server thread placeholder ==
_server_thread = None

# == Function to run the FastAPI server (in the background) ==
def run_server():
    config = uvicorn.Config(app, host='0.0.0.0', port=5000)  # Run on all interfaces (0.0.0.0), port 5000
    server = uvicorn.Server(config)
    server.run()

# == Start the server in a separate thread (non-blocking, good for Jupyter/Colab) ==
def start_server():
    global _server_thread
    if not _server_thread or not _server_thread.is_alive():
        _server_thread = threading.Thread(target=run_server, daemon=True)
        _server_thread.start()
        print("Lunched：http://localhost:5000/stream_chat")

# == Define a route for streaming chat ==
@app.get('/stream_chat')
async def stream_chat(param: str = "Hello"):
    """
    A GET endpoint that accepts a string parameter param and returns the streaming response of the language model.
    """
    async def generate():
        response_stream = query_engine.query(param)  
        for text in response_stream.response_gen:    
            yield text  # Use yield to generate each output segment to form a streaming response
    return StreamingResponse(generate(), media_type='text/event-stream')  # Returns SSE response, suitable for real-time display on web pages

# == Start the server (call it directly in Notebook) ==
start_server()


Lunched：http://localhost:5000/stream_chat


INFO:     Started server process [15120]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


### 4.3 Calling the FastAPI Streaming Endpoint (Client Side)

This example shows how to **send a request** to the `/stream_chat` endpoint we defined earlier using Python’s `requests` library.

The response is streamed back in **real time**, perfect for handling outputs from a chatbot or LLM that emits results token-by-token or line-by-line.

The code:
- Makes a `GET` request to the local FastAPI server
- Streams the response chunks gradually
- Prints them live to your terminal or notebook output


In [ ]:
# == Import requests library to send HTTP requests ==
import requests

# == Define a function to send a streaming request to FastAPI ==
def test_stream_chat(question="你好"):
    url = "http://localhost:5000/stream_chat"  # URL of the local FastAPI endpoint
    params = {"param": question}  # Query string parameter expected by the server
 
    # Make a streaming GET request
    with requests.get(url, params=params, stream=True) as response:
        # Iterate over the streamed content line-by-line
        for chunk in response.iter_content(decode_unicode=True):
            if chunk:
                print(chunk, end="", flush=True)  # Print without newline & flush buffer to show live text

# == Call the test function with an example query ==
test_stream_chat("What are the employment prospects for AI majors?")


INFO:     127.0.0.1:56836 - "GET /stream_chat?param=What+are+the+employment+prospects+for+AI+majors%3F HTTP/1.1" 200 OK
The provided information does not include specific details about employment prospects for AI majors.

INFO:     127.0.0.1:57028 - "GET /stream_chat?param=%E4%BD%A0%E6%98%AF%E8%B0%81%EF%BC%9F HTTP/1.1" 200 OK
INFO:     127.0.0.1:57028 - "GET /favicon.ico HTTP/1.1" 404 Not Found


## 5. From Backend to Frontend

We can create a python file (here `main.py`) with following code: 

In [15]:

# import uvicorn
# from fastapi import FastAPI
# from fastapi.middleware.cors import CORSMiddleware
# from fastapi.responses import StreamingResponse
# app = FastAPI()
# app.add_middleware(CORSMiddleware,allow_origins=["*"])
# @app.get('/stream_chat')
# async def stream_chat(param:str = "你好"):
#     def generate():  
#         # 我们假设query_engine已经构建完成
#         response_stream = query_engine.query(param) 
#         for text in response_stream.response_gen:
#             yield text
#     return StreamingResponse(generate(), media_type='text/event-stream')  
# if __name__ == '__main__':
#     uvicorn.run(app, host='0.0.0.0', port=5000)


The only difference between running this code in a Jupyter cell and running it in a `.py` file lies in the part after if `__name__ == '__main__'`:. This is because Jupyter is an interactive environment where code is executed cell by cell, rather than as a standalone program.

As a result, when we run a program in Jupyter, it runs in a new process instead of the main one. That's why in Jupyter, we need to use uvicorn.Server inside the if `__name__ == '__main__'`: block to manually start the server. However, this step is unnecessary when running the code from a regular Python file.

We can even open a browser and directly enter:

```http://127.0.0.1:5000/stream_chat?param=Who are you?```


the browser will then display the streaming output in real time.